In [20]:
import random
import re
import string
import numpy as np

In [73]:
file_path = "sentences.txt" 
sentences = []
word_encoder = {}
word_decoder = {}
counter = 0
with open(file_path, 'r') as file:
    for line in file:
        sentences.append(line.strip())  
for sentence in sentences:
    words = sentence.split()
    for word in words:
        word = word.lower()
        word = word.translate(str.maketrans('', '', string.punctuation))
        if len(word) and word not in word_encoder:
            word_encoder[word] = counter
            word_decoder[counter] = word
            counter = counter + 1

In [74]:
print("Number of sentence: ", len(sentences))
print("Number of unique words: ", counter)

Number of sentence:  7871825
Number of unique words:  1852140


**One word memory**

In [75]:
bigram = {}
count = 0

for sentence in sentences:
    words = sentence.split() 
    
    for i in range(len(words) - 1):
        first_word = words[i].lower()
        second_word = words[i+1].lower()
        first_word = first_word.translate(str.maketrans('', '', string.punctuation))
        second_word = second_word.translate(str.maketrans('', '', string.punctuation))
        if(len(first_word) and len(second_word)):
            current_bigram = (word_encoder[first_word],word_encoder[second_word])
            if current_bigram in bigram:
                bigram[current_bigram] += 1
            else:
                bigram[current_bigram] = 1
    count += 1
    if count % 500000 == 0:
        print(f"{count * 100 / len(sentences):.2f}%")


6.35%
12.70%
19.06%
25.41%
31.76%
38.11%
44.46%
50.81%
57.17%
63.52%
69.87%
76.22%
82.57%
88.92%
95.28%


In [76]:
bigram_count = bigram.get((word_encoder["the"],word_encoder["boy"]), 0)
print(bigram_count)

1000


In [120]:
def next_word_one(bigram, current_word):
    current_word_token = word_encoder[current_word]
    tokens_after = []
    for key in bigram:
        if key[0] == current_word_token:
            tokens_after.append(key[1])
    if len(tokens_after)==0:
        return 'a'
    unique_tokens_after = set(tokens_after)
    tokens_with_frequency = []
    total_freq = 0
    for token in unique_tokens_after:
        total_freq = total_freq +  bigram.get((current_word_token, token), 0)**1.2
    for token in unique_tokens_after:
        tokens_with_frequency.append((token,bigram.get((current_word_token, token), 0)**1.2/total_freq))
    sorted_tokens_with_frequency = sorted(tokens_with_frequency, key=lambda x: x[1], reverse=True)
    sorted_with_cum = []
    total_prob = 0
    for token in sorted_tokens_with_frequency:
        total_prob = total_prob + token[1]
        sorted_with_cum.append((token[0],total_prob))
    randi = random.uniform(0, 1)
    for token in sorted_with_cum:
        if(token[1]>=randi):
            return word_decoder[token[0]]


In [159]:
first_words = ["Basketball", "New", "Green", "Third", "Football", "School"]
for i, word in enumerate(first_words):
    print("Generated Sentence", i, ": ",end="")
    for i in range (0,20):
        print(word,end=" ")
        word = next_word_one(bigram,word.lower())
    print(".")

Generated Sentence 0 : Basketball player is a form a butterfly in california state of attacks that is a series first album was a .
Generated Sentence 1 : New york united states and the different language was used in the 1997 is an urban gospel of author and .
Generated Sentence 2 : Green patina formed in which it won by billboard magazine such as the wells and a star south africa the .
Generated Sentence 3 : Third largest release in the royal palace in berkshire england by the 1953 the juniors age income of the greek .
Generated Sentence 4 : Football player entertainer cher opened on 27 august 1971 is a dour except for the user will be found that .
Generated Sentence 5 : School at the 8 2012 the most important part of cleveland ohio united kingdom while not been replaced the pewter .


*We can notice how every word make sense relative to the word before it but not to the bigger picture of the sentence given the one word memory.*